In [25]:
import pandas as pd
import xlwings as xw
from tqdm import tqdm
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import sys
sys.path.append('../')
from data.jet_sim import *

In [26]:
def data_nplist(file_path:str = '../data/CFD0.8_0.2.csv',columns_index = 0,is_mean = True):
    pd_data = pd.read_csv(file_path)
    j = pd_data.iloc[0,0]
    num = 0
    cut_list = [0]
    for i in pd_data.iloc[:,0]:
        if abs(j-i) > 1:
            j = i
            cut_list.append(num)
        else:
            j = i
        num += 1
    cut_list.append(num-1)

    res = dict()
    for i in range(len(cut_list)-1):
        res[f'ID{i}'] = pd_data.iloc[cut_list[i]:cut_list[i+1],]

    min_num = 10000
    for i in range(len(res)):
        if len(res[f'ID{i}']) < min_num:
            min_num = len(res[f'ID{i}'])

    np_list = np.empty((min_num,0))
    for i in res.keys():
        b = np.asarray(res[i].iloc[:min_num,columns_index]).reshape(-1,1)
        np_list = np.concatenate((np_list,b),axis=1)
    if is_mean:
        np_list = np_list.mean(axis=1)
        np_list = np_list - np_list.min() # 坐标归零
    return(np_list)

def reslut_plot(TIME,DELTA_T,K,U,EXHALE,file_path='../data/CFD0.8_0.2.csv',name = ['sim','cfd']):
    sim_data = transform_pd(sim_jet(TIME,DELTA_T,K,U,EXHALE))

    cfd_data_x = data_nplist(file_path,columns_index = 0)
    cfd_data_y = data_nplist(file_path,columns_index = 1)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sim_data['x'], y=sim_data['y'], mode='lines', name=name[0]))
    fig.add_trace(go.Scatter(x=cfd_data_x, y=cfd_data_y, mode='lines', name=name[1]))
    fig.update_xaxes(range=[0, 2])
    fig.update_yaxes(range=[0, 0.3])
    fig.update_layout(title=f'Attenuation coefficient={K},\
                    <br />Ambient airflow={U},Exhale={EXHALE[1]}', xaxis_title='x(m)', yaxis_title='y(m)',
                    font_family = 'Times New Roman',width=800, height=600)
    return fig

In [27]:
TIME = 20
DELTA_T = 0.005
K = 0.5
U = [0.2,0.4,0.6]
EXHALE = [0.55,0.8,1.1]
def output(U,E,name = ['Calc','Sim']):
    FILE_PATH = f'../data/{E}_{U}.csv'
    return reslut_plot(TIME,DELTA_T,K,U,[0,E],FILE_PATH,name = name)